# Chapitre 5 : Segmentation

Ce notebook a pour objectif de vous montrer deux approches de segmentation d'images: 

- approche basée sur le seuillage
- approche basée sur la classification

Les données de ce notebook proviennent du site http://medicaldecathlon.com/


## Exercice 1: 

1) Lisez les images "BRATS_001.nii" et "BRATS_001_tumor_mask.nii"

In [1]:
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt

inputVolume = nib.load("data/BRATS_001.nii")
data = inputVolume.get_fdata()

inputMask = nib.load("data/BRATS_001_tumor_mask.nii")
mask_data =  inputMask.get_fdata()

2) A l'aide de la fonction "contrast_histogram_and_inside_mask" qui vous a été fournie dans le corrigé de l'exercice1, affichez l'histogramme à l'intérieur de la tumeur globale en même temps que l'histogramme global 3D de l'image. Le curseur vous permettra de naviguer entre les différents contraste et de voir l'allure de l'histogramme à l'intérieur de la tumeur par rapport à l'histogramme de l'image pour chaque contraste. Choisissez le contraste pour lequel les niveaux de gris de la tumeur semblent les mieux isolés. Choisissez un seuil adéquat et effectuez une segmentation de la tumeur globale par seuillage. Sauvegardez le résultat sous le nom "BRATS_001_whole_tumor_segmentation.nii"


In [2]:
from ipywidgets import interact,fixed
def contrast_histogram_and_inside_mask(input_volume,mask_volume,contrast_index):

    data_array = input_volume[:,:,:,contrast_index].flatten()
    min_data = np.array(data_array).min()
    max_data = np.array(data_array).max()
    plt.figure(figsize=(10, 5))
    plt.hist(data_array,range(int(min_data), int(max_data),5),label=['whole histogram'],color ='b')
        
    inputSizeX = mask_volume.shape[0]
    inputSizeY = mask_volume.shape[1]
    inputSizeZ = mask_volume.shape[2]
    data_list = []

    for z in range( inputSizeZ ):
        for y in range( inputSizeY ):
            for x in range( inputSizeX ):
                if( mask_volume[ x, y, z ] != 0 ):
                    data_list.append(input_volume[x,y,z,contrast_index])

    mask_data_array = np.array(data_list)
    plt.hist(mask_data_array,range(int(min_data), int(max_data),5),label=['histogram inside mask'],color ='g')
    plt.legend(loc='upper right')    
    plt.ylim(0, 60000)
    plt.show()
    
interact(contrast_histogram_and_inside_mask, input_volume=fixed(data), mask_volume=fixed(mask_data),contrast_index=(0, 3));

interactive(children=(IntSlider(value=1, description='contrast_index', max=3), Output()), _dom_classes=('widge…

In [3]:
seuil=750
contraste=0
inputSizeX = data.shape[0]
inputSizeY = data.shape[1]
inputSizeZ = data.shape[2]

data_seuil=np.zeros((inputSizeX, inputSizeY, inputSizeZ))
for z in range( inputSizeZ ):
    for y in range( inputSizeY ):
        for x in range( inputSizeX ):
            if( data[ x, y, z,contraste ] <= seuil ):
                data_seuil[x,y,z]=0
            else:
                data_seuil[x,y,z]=1

In [4]:
from ipywidgets import interact,fixed
import matplotlib.pyplot as plt

def explore_3dimage_axial(layer, input_data):
    plt.figure(figsize=(10, 5))
    plt.imshow(input_data[:, :, layer], cmap='gray');
    plt.title('Mask', fontsize=20)
    plt.axis('off')
    return layer


interact(explore_3dimage_axial, layer=(0, data_seuil.shape[2] - 1),input_data=fixed(data_seuil));

interactive(children=(IntSlider(value=77, description='layer', max=154), Output()), _dom_classes=('widget-inte…

In [5]:
img = nib.Nifti1Image(data_seuil, np.eye(4))  

img.header.get_xyzt_units()
img.to_filename("BRATS_001_whole_tumor_segmentation.nii")  

3) Lisez l'image "BRATS_001_tumor_labels.nii". Développez une nouvelle fonction qui permet d'afficher en même temps l'histogramme à l'intérieur de la tumeur globale ainsi que l'histogramme à l'intérieur de chaque sous région. Vous créerez pour cela un autre curseur qui vous permet de naviguer entre les sous régions en plus du curseur qui vous permet de naviger entre les contrastes.

In [6]:
inputVolume = nib.load("data/BRATS_001_tumor_labels.nii")
data2 = inputVolume.get_fdata()

inputVolume = nib.load("data/BRATS_001.nii")
data = inputVolume.get_fdata()

inputMask = nib.load("data/BRATS_001_tumor_mask.nii")
mask_data =  inputMask.get_fdata()

In [7]:
from ipywidgets import interact,fixed
def contrast_histogram_and_inside_mask(input_volume,mask_volume,label_volume,contrast_index):

    plt.figure(figsize=(10, 5))
    #plt.hist(data_array,range(int(min_data), int(max_data),5),label=['whole histogram'],color ='b')
        
    inputSizeX = label_volume.shape[0]
    inputSizeY = label_volume.shape[1]
    inputSizeZ = label_volume.shape[2]
    data_list0 = []
    data_list1 = []
    data_list2 = []
    data_list3 = []

    for z in range( inputSizeZ ):
        for y in range( inputSizeY ):
            for x in range( inputSizeX ):
                if( mask_volume[ x, y, z ] != 0 ):
                    data_list0.append(input_volume[x,y,z,contrast_index])

                if( label_volume[ x, y, z ] == 1 ):
                    data_list1.append(input_volume[x,y,z,contrast_index])
                elif( label_volume[ x, y, z ] == 2 ):
                    data_list2.append(input_volume[x,y,z,contrast_index])
                elif( label_volume[ x, y, z ] == 3 ):
                    data_list3.append(input_volume[x,y,z,contrast_index])

    
    mask_data_array = np.array(data_list0)
    plt.hist(mask_data_array,range(0, 2000,5),label=['histogram inside mask'],color ='g')
    
    mask_data_array = np.array(data_list1)
    plt.hist(mask_data_array,range(0, 2000,5),label=['label 1'],color ='r')
    
    mask_data_array = np.array(data_list2)
    plt.hist(mask_data_array,range(0, 2000,5),label=['label 2'],color ='b')
    
    mask_data_array = np.array(data_list3)
    plt.hist(mask_data_array,range(0, 2000,5),label=['label 3'],color ='y')
    
    
    plt.legend(loc='upper right')    
    plt.ylim(0, 6000)
    plt.show()
    
interact(contrast_histogram_and_inside_mask, input_volume=fixed(data), mask_volume=fixed(mask_data),label_volume=fixed(data2),contrast_index=(0, 3));

interactive(children=(IntSlider(value=1, description='contrast_index', max=3), Output()), _dom_classes=('widge…

4) Choisissez les bons seuils et les bons contrastes à partir desquels le seuillage sera effectué à l'intérieur du masque global de la tumeur de façon à obtenir une segmentation des sous régions qui s'approche le plus possible de la vérité terrain fourie par l'image "BRATS_001_tumor_labels.nii".

In [8]:
data_seuil=np.zeros((inputSizeX, inputSizeY, inputSizeZ))

seuil1=600
seuil2=1000
contraste=2
inputSizeX = data.shape[0]
inputSizeY = data.shape[1]
inputSizeZ = data.shape[2]

data_seuil=np.zeros((inputSizeX, inputSizeY, inputSizeZ))
for z in range( inputSizeZ ):
    for y in range( inputSizeY ):
        for x in range( inputSizeX ):
            if( data[ x, y, z,contraste ] <= seuil1 ):
                data_seuil[x,y,z]=2
            elif( seuil1 < data[ x, y, z,contraste ] <= seuil2 ):
                data_seuil[x,y,z]=1
            elif( seuil2 < data[ x, y, z,contraste ]):
                data_seuil[x,y,z]=3
            

In [9]:
img = nib.Nifti1Image(data_seuil, np.eye(4))  

img.header.get_xyzt_units()
img.to_filename("BRATS_001_tumor_labels.nii")  

## Exercice 2: 

1) Utilisez les images "BRATS_001.nii" et "BRATS_001_tumor_mask.nii" pour travailler uniquement sur les niveaux de gris à l'intérieur du masque global de la tumeurs. A l'aide de la méthode de classification k means et en utilisant les 4 volumes de contrastes, classifiez l'intérieur de la tumeur en 3 sous régions.

In [10]:
inputVolume = nib.load("data/BRATS_001.nii")
data = inputVolume.get_fdata()

inputMask = nib.load("data/BRATS_001_tumor_mask.nii")
mask_data =  inputMask.get_fdata()

inputSizeX = data.shape[0]
inputSizeY = data.shape[1]
inputSizeZ = data.shape[2]
inputSizeC = data.shape[3]

for c in range(inputSizeC):
    for z in range( inputSizeZ ):
        for y in range( inputSizeY ):
            for x in range( inputSizeX ):
                if(mask_data[x, y, z]>0):
                    data[ x, y, z, c ] = data[ x, y, z, c ]
                else:
                    data[x,y,z, c]=1
                

In [11]:
from ipywidgets import interact,fixed
import matplotlib.pyplot as plt

def explore_3dimage_axial(layer1, layer2, input_data):
    plt.figure(figsize=(10, 5))
    plt.imshow(input_data[:, :, layer1, layer2], cmap='gray');
    plt.title('Apply mask', fontsize=20)
    plt.axis('off')
    return layer1, layer2


interact(explore_3dimage_axial, layer1=(0, data.shape[2] - 1),
         layer2=(0, data.shape[3] - 1),input_data=fixed(data));

interactive(children=(IntSlider(value=77, description='layer1', max=154), IntSlider(value=1, description='laye…

In [14]:
from sklearn.cluster import KMeans

all_img=[]

inputSizeX = data.shape[0]
inputSizeY = data.shape[1]
for C in range(data.shape[3]):
    img=[]
    for Z in range(data.shape[2]):
        data2 = data[:,:,Z,C].reshape(-1, 1)
        kmeans = KMeans(n_clusters=3, random_state=0)
        kmeans.fit(data2)
        centers = kmeans.cluster_centers_
        labels = kmeans.labels_
        img.append(centers[labels].reshape(inputSizeX, inputSizeY))

    all_img.append(img)
    

C:\Users\admin\anaconda3\lib\site-packages\ipykernel_launcher.py:12: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (3). Possibly due to duplicate points in X.
  if sys.path[0] == '':
C:\Users\admin\anaconda3\lib\site-packages\ipykernel_launcher.py:12: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (3). Possibly due to duplicate points in X.
  if sys.path[0] == '':
C:\Users\admin\anaconda3\lib\site-packages\ipykernel_launcher.py:12: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (3). Possibly due to duplicate points in X.
  if sys.path[0] == '':
C:\Users\admin\anaconda3\lib\site-packages\ipykernel_launcher.py:12: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (3). Possibly due to duplicate points in X.
  if sys.path[0] == '':
C:\Users\admin\anaconda3\lib\site-packages\ipykernel_launcher.py:12: ConvergenceWarning: Number of distinct clusters (1) fou

In [15]:
all_img=np.array(all_img)
all_img.shape

(4, 155, 240, 240)

In [18]:
from ipywidgets import interact,fixed
import matplotlib.pyplot as plt

def explore_3dimage_axial(contraste, layer, input_data):
    plt.figure(figsize=(10, 5))
    plt.imshow(input_data[contraste,layer, :, :], cmap='gray');
    plt.title('Segmented', fontsize=20)
    plt.axis('off')
    return contraste, layer


interact(explore_3dimage_axial, contraste=(0, all_img.shape[0] - 1),
         layer=(0, all_img.shape[1] - 1), input_data=fixed(all_img));

interactive(children=(IntSlider(value=1, description='contraste', max=3), IntSlider(value=77, description='lay…

2) Sauvegardez l'image d'étiquette obtenue sous le nom "BRATS_001_tumor_kmeans_segmentation.nii"

In [19]:
img = nib.Nifti1Image(all_img, np.eye(4))  

img.header.get_xyzt_units()
img.to_filename("BRATS_001_tumor_kmeans_segmentation.nii")  